# K Nearest Neighbors

Практическая работа по методу k-ближайших соседей.

## Загрузка библиотек



In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

sns.set_style('whitegrid')
%matplotlib inline

## Загрузка данных

Данные взяты с сайта Kaggle.com: https://www.kaggle.com/abcsds/pokemon

In [ ]:
data = pd.read_csv("data/Pokemon.csv", index_col=0).reset_index(drop=True)

In [ ]:
data.head()

# Exploratory Data Analysis - описательное исследование данных

Начнем рассмотрение данных с пропущенных данных, затем посмотрим на распределение категориальных и числовых данных.

## Missing Data - пропущенные данные

In [ ]:
sns.heatmap(data.isnull(), yticklabels=False, cbar=False, cmap='viridis')
plt.show()

В наших данных мало пропусков - всего одна колонка. 

Есть несколько способов бороться с пропусками в категориальных данных:
* Удалить такие колонки;
* Заполнить новыми значениями, тем самым создав дополнительную группу.

Поступим вторым способом.

In [ ]:
data.loc[data['Type 2'].isnull(), 'Type 2'] = 'NoneType'

## Распределения

Посмотрим на таргет, а потом на остальные колонки.

In [ ]:
fig = sns.countplot(x='Legendary', data=data)
plt.show()

Виден сильный перекос в таргете, поэтому придется с осторожностью подойти к подбору метрики качества.

Ниже распределения остальных категориальных фич и числовых.

In [ ]:
fig = sns.countplot(x='Generation', data=data)
plt.show()

In [ ]:
fig = sns.countplot(x='Generation', hue='Legendary', data=data)

plt.show()

In [ ]:
fig = sns.countplot(x='Type 1', hue='Legendary', data=data)
fig.set_xticklabels(fig.get_xticklabels(), rotation=70)
plt.show()

In [ ]:
fig = sns.countplot(x='Type 2', hue='Legendary', data=data)
fig.set_xticklabels(fig.get_xticklabels(), rotation=70)
plt.show()

In [ ]:
num_cols = [col for col in data.columns if data[col].dtype == 'float64' or data[col].dtype == 'int64']
need_cols = num_cols+['Legendary']

In [ ]:
sns.pairplot(data[need_cols], hue='Legendary')
plt.show()

In [ ]:
for col in num_cols:
    sns.distplot(data[col])
    plt.show()

## Преобразуем  категориальные данные в числовые.

В метрических алгоритмах есть так называемое проклятье размерности. Это когда размерность пространства признаков настолько большая, что для алгоритма все объекты равноудалены и классификация или регрессия по ним затруднительна. 

Поэтому при преобразовании категориальных признаков нам не хотелось бы сильно увеличивать размерность пространства признаков.

Есть два основынх способа преобразования категориальных переменных:
* Дамми-переменные, когда каждому значению категориальной переменной соответствует своя колонка и единичка ставится в соответствующую колонку, если у объекта данное значение категориального признака. Если в переменной 3 значения, то колонок тоже будет три, и 1 будут стоять в соответствующий колонке.
* Каждому значению категориальной переменной присвоить свое число. Самый простой способ - это LabelEncoding.

Для нас лучше начать с LabelEncoding, так как таким способом новых фичей мы не будем добавлять. А в случае с дамми переменными пришлось бы еще много колонок добавить. 

In [ ]:
data.info()

Всего у нас 3 нечисловые фичи, помимо таргета, который легко преобразуется в 0 и 1. Но с Name мы не будем сейчас разбираться. Хотя самые интересующуюся могут применить к Name методы NLP. Например, TF-IDF. Поэтому пока просто выкинем Name.

In [ ]:
data.drop('Name', inplace=True, axis=1)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
data['Type 1'] = encoder.fit_transform(data['Type 1'])
data['Type 2'] = encoder.fit_transform(data['Type 2'])

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, data['Legendary'],
                                                    test_size=0.3, random_state=42)

## Скалирование признаков.

Метрические методы крайне чувствительны к масштабу признаков, так как большим по масштабу признакам будет придано большее значение, ведь они вносят больший вклад в совокупное расстояние. Хотя, на самом деле, они могут быть и не важны. Поэтому хорошо было бы привести их к одному порядку.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
num_cols = [col for col in X_train.columns if X_train[col].dtype == 'float64' or X_train[col].dtype == 'int64']

In [ ]:
scaler.fit(X_train[num_cols])

In [ ]:
scaled_features = scaler.transform(X_train[num_cols])
X_test = scaler.transform(X_test[num_cols])

In [ ]:
df_feat = pd.DataFrame(scaled_features, columns=num_cols)
X_test = pd.DataFrame(X_test, columns=num_cols)
df_feat.head()

## Построение модели KNN

Начнем с простого случая одного соседа.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)

In [ ]:
knn.fit(df_feat, y_train)

In [ ]:
pred = knn.predict_proba(X_test)[:, 1]

## Предсказание и оценка модели

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score

In [ ]:
print(confusion_matrix(y_test, pred))

In [ ]:
print('Precision (class False) {:.2f} Recall (class False) {:.2f}\
      \nPrecision (class True)  {:.2f} Recall (calss True)  {:.2f}'.format(218/223, 218/225, 10/17, 10/15))

In [ ]:
print(classification_report(y_test, pred))

In [ ]:
print('{:.2f} {:.2f}'.format((218+10)/(218+10+7+5), accuracy_score(y_test, pred)))

In [ ]:
print('{:.2f}'.format(roc_auc_score(y_test, pred)))

## Выбор K

In [ ]:
error_rate = []

for i in range(1, 40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(df_feat, y_train)
    pred_i = knn.predict_proba(X_test)[:, 1]
    error_rate.append(roc_auc_score(y_test, pred_i))

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, 40),error_rate, color='blue', linestyle='dashed', marker='.')
plt.title('ROC-AUC vs. K Value')
plt.xlabel('K')
plt.ylabel('ROC-AUC')
errors = list(zip(range(1,100), error_rate))
errors = sorted(errors, key=lambda tup: tup[1], reverse=True)
vert_line = errors[0][0]
plt.vlines(x=vert_line, ymin=min(error_rate), ymax=max(error_rate)+0.005, color='red', linestyles='dashed')
plt.show()

In [ ]:
errors = list(zip(range(1,100), error_rate))
errors = sorted(errors, key=lambda tup: tup[1], reverse=True)
errors[:5]

## Cross Validation

In [ ]:
target = data['Legendary']*1

In [ ]:
from sklearn.model_selection import KFold
error_rate = []

for i in range(1, 50):
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    val_rate = []
    for tr_ind, val_ind in kf.split(data):
        knn = KNeighborsClassifier(n_neighbors=i)
        train = data.loc[tr_ind, num_cols]
        val = data.loc[val_ind, num_cols]
        scaler = MinMaxScaler()
        scaler.fit(train)
        train = scaler.transform(train)
        val = scaler.transform(val)
        train = pd.DataFrame(train, columns=num_cols)
        val = pd.DataFrame(val, columns=num_cols)
        
        target_train = target[tr_ind]
        target_val = target[val_ind]

        knn.fit(train, target_train)
        pred_i = knn.predict_proba(val)[:, 1]
        val_rate.append(roc_auc_score(target_val, pred_i))
    error_rate.append(np.mean(val_rate))

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(range(1, 50),error_rate, color='blue', linestyle='dashed', marker='.')
plt.title('{}, {:.3f}'.format('ROC-AUC vs. K Value', max(error_rate)))
plt.xlabel('K')
plt.ylabel('ROC-AUC')
errors = list(zip(range(1, 50), error_rate))
errors = sorted(errors, key=lambda tup: tup[1], reverse=True)
vert_line = errors[0][0]
plt.vlines(x=vert_line, ymin=min(error_rate), ymax=max(error_rate)+0.005, color='red', linestyles='dashed')
plt.show()

In [ ]:
metrics = ['euclidean', 'manhattan', 'chebyshev', 'minkowski']
for metric in metrics:
    error_rate = []
    for i in range(1, 50):
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        val_rate = []
        for tr_ind, val_ind in kf.split(data):
            knn = KNeighborsClassifier(n_neighbors=i, metric=metric)
            train = data.loc[tr_ind, num_cols]
            val = data.loc[val_ind, num_cols]
            scaler = MinMaxScaler()
            scaler.fit(train)
            train = scaler.transform(train)
            val = scaler.transform(val)
            train = pd.DataFrame(train, columns=num_cols)
            val = pd.DataFrame(val, columns=num_cols)

            target_train = target[tr_ind]
            target_val = target[val_ind]

            knn.fit(train, target_train)
            pred_i = knn.predict_proba(val)[:, 1]
            val_rate.append(roc_auc_score(target_val, pred_i))
        error_rate.append(np.mean(val_rate))
        
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, 50), error_rate, color='blue', linestyle='dashed', marker='.')
    plt.title('{}, {}, {:.3f}'.format('ROC-AUC vs. K Value', metric, max(error_rate)))
    plt.xlabel('K')
    plt.ylabel('ROC-AUC')
    errors = list(zip(range(1, 50), error_rate))
    errors = sorted(errors, key=lambda tup: tup[1], reverse=True)
    vert_line = errors[0][0]
    plt.vlines(x=vert_line, ymin=min(error_rate), ymax=max(error_rate)+0.005, color='red', linestyles='dashed')
    plt.show()

# Домашнее задание

Вам необходимо  построить модель по кросс-валидации с максимальном скором метрики roc_auc.

Ограничения: не более 50 соседей, не более 10 фолдов и фиксированный random_state, то есть:
>     for i in range(1, 50):
>         kf = KFold(n_splits=10, shuffle=True, random_state=42)

Советы для возможных улучшений:
* Перебирать параметры для меделей, например, можно подобрать оптимальный p для метрики minkowski
* Удаление или генерация колонок тоже может привести к росту скора.

# Отличная работа!

Не ленитесь пробовать новое!